In [57]:
# import, configuration, functions, etc.
# Run this every time you use the script

import json
import requests
from time import sleep
import csv
import datetime

def generateHeaderDictionary(acceptMediaType):
    userAgentHeader = 'ctgBot/0.1 (https://github.com/HeardLibrary/linked-data/tree/master/clinical_trials; mailto:steve.baskauf@vanderbilt.edu)'
    requestHeaderDictionary = {
        'Accept' : acceptMediaType,
        'User-Agent': userAgentHeader
    }
    return requestHeaderDictionary

def get_ctg(parameter, result_start, result_end):
    endpointUrl = 'https://clinicaltrials.gov/api/query/full_studies'
    acceptMediaType = 'application/json'
    
    parameters = {
        'expr': parameter,
        'min_rnk': result_start,
        'max_rnk': result_end,
        'fmt': 'json'
    }
    
    r = requests.get(endpointUrl, params = parameters, headers = generateHeaderDictionary(acceptMediaType))
    data = r.json()
    return data



In [58]:
# API home https://clinicaltrials.gov/api/gui/home
# for search expression syntax, see https://clinicaltrials.gov/api/gui/ref/syntax 
# for field name crosswalks, see https://clinicaltrials.gov/api/gui/ref/crosswalks
search_parameter = 'SEARCH[Study]AREA[ResponsiblePartyInvestigatorAffiliation]COVERAGE[Contains] Vanderbilt'
result_start = 26
result_end = 30
data = get_ctg(search_parameter, result_start, result_end)
print(json.dumps(data, indent=2))
study_list = data['FullStudiesResponse']['FullStudies']



{
  "FullStudiesResponse": {
    "APIVrs": "1.01.02",
    "DataVrs": "2020:05:04 23:59:49.930",
    "Expression": "SEARCH[Study]AREA[ResponsiblePartyInvestigatorAffiliation]COVERAGE[Contains] Vanderbilt",
    "NStudiesAvail": 338266,
    "NStudiesFound": 1069,
    "MinRank": 26,
    "MaxRank": 30,
    "NStudiesReturned": 5,
    "FullStudies": [
      {
        "Rank": 26,
        "Study": {
          "ProtocolSection": {
            "IdentificationModule": {
              "NCTId": "NCT02984709",
              "OrgStudyIdInfo": {
                "OrgStudyId": "161732"
              },
              "Organization": {
                "OrgFullName": "Vanderbilt University Medical Center",
                "OrgClass": "OTHER"
              },
              "BriefTitle": "Check It! 2.0: Positive Psychology Intervention for Adolescents With Type 1 Diabetes",
              "OfficialTitle": "Check It! 2.0: Pilot Test of a Positive Psychology Intervention for Adolescents With Type 1 Diabetes"
   

In [59]:
for study in study_list:
    study_dict = study['Study']
    NCTId = study_dict['ProtocolSection']['IdentificationModule']['NCTId']
    pi_name = study_dict['ProtocolSection']['SponsorCollaboratorsModule']['ResponsibleParty']['ResponsiblePartyInvestigatorFullName']
    pi_title = study_dict['ProtocolSection']['SponsorCollaboratorsModule']['ResponsibleParty']['ResponsiblePartyInvestigatorTitle']

    #print(study_dict['ProtocolSection'])
    contacts = study_dict['ProtocolSection']['ContactsLocationsModule']
    if 'OverallOfficialList' in contacts:
        officials = contacts['OverallOfficialList']['OverallOfficial']
    else:
        officials = []
    print(NCTId)
    print(pi_name)
    print(pi_title)
    print(officials)
    print()


NCT02984709
Sarah Jaser
Assistant Professor
[{'OverallOfficialName': 'Sarah Jaser, PhD', 'OverallOfficialAffiliation': 'Vanderbilt University Medical Center', 'OverallOfficialRole': 'Principal Investigator'}]

NCT02979353
Sandra Simmons
Principal Investigator
[{'OverallOfficialName': 'Sandra F Simmons, PhD', 'OverallOfficialAffiliation': 'Vanderbilt University Medical Center', 'OverallOfficialRole': 'Principal Investigator'}]

NCT02975583
Joshua Beckman
Professor of Medicine
[{'OverallOfficialName': 'Joshua A Beckman', 'OverallOfficialAffiliation': 'Vanderbilt University Medical Center', 'OverallOfficialRole': 'Principal Investigator'}]

NCT02971202
Justin Gregory
Assistant Professor of Pediatrics
[{'OverallOfficialName': 'Justin M Gregory, MD', 'OverallOfficialAffiliation': 'Vanderbilt University School of Medicine', 'OverallOfficialRole': 'Principal Investigator'}]

NCT02964884
Laura Cutting
Principal Investigator
[{'OverallOfficialName': 'Laurie Cutting, PhD', 'OverallOfficialAffili